<h1 align="center"> TUGAS BESAR TF3101 - DINAMIKA SISTEM DAN SIMULASI </h1>
<h2 align="center"> Sistem Elektrik, Elektromekanik, dan Termofluidik</h2>

<h3>Nama Anggota:</h3>
<body>
    <ul>
        <li>Erlant Muhammad Khalfani (13317025)</li>
        <li>Bernardus Rendy (13317041)</li>
    </ul>
</body>

## 1. Pemodelan Sistem Elektrik##

Untuk pemodelan sistem elektrik, dipilih rangkaian RLC seri dengan sebuah sumber tegangan seperti yang tertera pada gambar di bawah ini.

<img src="./ELEKTRIK_TUBES_3.png" style="width:50%" align="middle">

### Deskripsi Sistem

1. Input <br>
Sistem ini memiliki input sumber tegangan $v_i$, yang merupakan fungsi waktu $v_i(t)$. <br>
2. Output <br>
Sistem ini memiliki *output* arus $i_2$, yaitu arus yang mengalir pada *mesh* II. Tegangan $v_{L1}$ dan $v_{R2}$ juga dapat berfungsi sebagai *output*. Pada program ini, *output* yang akan di-*plot* hanya $v_{R2}$ dan $v_{L1}$. Nilai $i_2$ berbanding lurus terhadap nilai $v_{R2}$, sehingga bentuk grafik $i_2$ akan menyerupai bentuk grafik $v_{R2}$
3. Parameter <br>
Sistem ini memiliki parameter-parameter $R_1$, $R_2$, $L_1$, dan $C_1$. Hambatan-hambatan $R_1$ dan $R_2$ adalah parameter *resistance*. Induktor $L_1$ adalah parameter *inertance*. Kapasitor $C_1$ adalah parameter *capacitance*.

### Asumsi
1. Arus setiap *mesh* pada keadaan awal adalah nol ($i_1(0) = i_2(0) = 0$).
2. Turunan arus terhadap waktu pada setiap *mesh* adalah nol ($\frac{di_1(0)}{dt}=\frac{di_2(0)}{dt}=0$)

### Pemodelan dengan *Bond Graph*

Dari sistem rangkaian listrik di atas, akan didapatkan *bond graph* sebagai berikut.
<img src="./BG_ELEKTRIK.png" style="width:50%" align="middle">
<br>
Pada gambar di atas, terlihat bahwa setiap *junction* memenuhi aturan kausalitas. Ini menandakan bahwa rangkaian di atas bersifat *causal*. Dari *bond graph* di atas, dapat diturunkan *Ordinary Differential Equation* (ODE) seperti hasil penerapan *Kirchhoff's Voltage Law* (KVL) pada setiap *mesh*. Dalam pemodelan *bond graph* variabel-variabel dibedakan menjadi variabel *effort* dan *flow*. Sistem di atas merupakan sistem elektrik, sehingga memiliki variabel *effort* berupa tegangan ($v$) dan *flow* berupa arus ($i$).

### Persamaan Matematis - ODE
Dilakukan analisis besaran *effort* pada *1-junction* sebelah kiri. Ini akan menghasilkan:
$$
v_i = v_{R1} + v_{C1}
$$
<br>
Hasil ini sama seperti hasil dari KVL pada *mesh* I. Nilai $v_{R1}$ dan $v_{C1}$ diberikan oleh rumus-rumus:
$$
v_{R1} = R_1i_1
$$
<br>
$$
v_{C1} = \frac{1}{C_1}\int (i_1 - i_2)dt
$$
sehingga hasil KVL pada *mesh* I menjadi:
$$
v_i = R_1i_1 + \frac{1}{C_1}\int (i_1 - i_2)dt
$$

Kemudian, analisis juga dilakukan pada *1-junction* sebelah kanan, yang akan menghasilkan:
$$
v_{C1} = v_{R2} + v_{L1}
$$
<br>
Ini juga sama seperti hasil KVL pada *mesh* II. Nilai $v_{R2}$ dan $v_{L1}$ diberikan oleh rumus-rumus:
$$
v_{R2} = R_2i_2
$$
<br>
$$
v_{L1} = L_1\frac{di_2}{dt}
$$
sehingga hasil KVL pada *mesh* II menjadi:
$$
\frac{1}{C_1}\int(i_1-i_2)dt = R_2i_2 + L_1\frac{di_2}{dt}
$$
atau
$$
0 = L_1\frac{di_2}{dt} + R_2i_2 + \frac{1}{C_1}\int(i_2-i_1)dt
$$

### Persamaan Matematis - *Transfer Function*
Setelah didapatkan ODE hasil dari *bond graph*, dapat dilakukan *Laplace Transform* untuk mendapatkan fungsi transfer sistem. *Laplace Transform* pada persamaan hasil KVL *mesh* I menghasilkan:
$$
(R_1 + \frac{1}{C_1s})I_1 + (-\frac{1}{C_1s})I_2 = V_i
$$
<br>
dan pada persamaan hasil *mesh* II, akan didapatkan:
$$
(-\frac{1}{C_1s})I_1 + (L_1s + R_2 + \frac{1}{C_1s})I_2 = 0
$$
<br>
Kedua persamaan itu dieliminasi, sehingga didapatkan fungsi transfer antara $I_2$ dengan $V_i$
$$
\frac{I_2(s)}{V_i(s)} = \frac{1}{R_1C_1L_1s^2 + (R_1R_2C_1 + L_1)s + R_1 + R_2}
$$
<br>
Dari hasil *Laplace Transform* persamaan pada *mesh* II, didapatkan nilai $V_{L1}$ dari rumus
$$
V_{L1} = L_1sI_2
$$
<br>
sehingga didapatkan fungsi transfer antara $V_{L1}$ dengan $V_i$
$$
\frac{V_{L1}(s)}{V_i(s)} = \frac{L_1s}{R_1C_1L_1s^2 + (R_1R_2C_1 + L_1)s + R_1 + R_2}
$$
Sementara fungsi transfer antara $V_{R2}$ dan $V_i$ adalah
$$
\frac{V_{R2}(s)}{V_i(s)} = \frac{R_2}{R_1C_1L_1s^2 + (R_1R_2C_1 + L_1)s + R_1 + R_2}
$$

In [1]:
#IMPORTS
from ipywidgets import interact, interactive, fixed, interact_manual , HBox, VBox, Label, Layout
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

In [2]:
#DEFINISI SLIDER-SLIDER PARAMETER
#Slider R1
R1_slider = widgets.FloatSlider(
    value=1.,
    min=1.,
    max=1000.,
    step=1.,
    description='$R_1 (\Omega)$',
    readout_format='.1f',
)
#Slider R2
R2_slider = widgets.FloatSlider(
    value=1.,
    min=1.,
    max=1000.,
    step=1.,
    description='$R_2 (\Omega)$',
    readout_format='.1f',
)
#Slider C1
C1_slider = widgets.IntSlider(
    value=1,
    min=10,
    max=1000,
    step=1,
    description='$C_1 (\mu F)$',
)
#Slider L1
L1_slider = widgets.FloatSlider(
    value=0.1,
    min=1.,
    max=1000.,
    step=0.1,
    description='$L_1 (mH)$',
    readout_format='.1f',
)

In [3]:
#DEKLARASI SELECTOR INPUT
#Slider selector input
vi_select = signal_select = widgets.Dropdown(
    options=[('Step', 0), ('Impulse', 1)],
    description='Tipe Sinyal:',
)
#DEKLARASI SELECTOR OUTPUT
#Output Selector
vo_select = widgets.ToggleButtons(
    options=['v_R2', 'v_L1'],
    description='Output:',
)

In [4]:
#DEKLARASI TAMBAHAN UNTUK INTERFACE
#Color button
color_select1 = widgets.ToggleButtons(
    options=['blue', 'red', 'green', 'black'],
    description='Color:',
)

In [5]:
#PENENTUAN NILAI-NILAI PARAMETER
R1 = R1_slider.value
R2 = R2_slider.value
C1 = C1_slider.value
L1 = L1_slider.value

#PENENTUAN NILAI DAN BENTUK INPUT
vform = vi_select.value
#PENENTUAN OUTPUT
vo = vo_select

#PENENTUAN PADA INTERFACE
color = color_select1.value

In [6]:
#Plot v_L1 menggunakan transfer function
def plot_electric (vo, R1, R2, C1, L1, vform, color):
    #Menyesuaikan nilai parameter dengan satuan
    R1 = R1
    R2 = R2
    C1 = C1*(10**-6)
    L1 = L1*(10**-3)
    
    f, ax = plt.subplots(1, 1, figsize=(8, 6))
    num1 = [R2]
    num2 = [L1, 0]
    den = [R1*C1*L1, R1*R2*C1+L1, R1+R2]
    if vo=='v_R2':
        sys_vr =signal.TransferFunction(num1, den)
        step_vr = signal.step(sys_vr)
        impl_vr = signal.impulse(sys_vr)
        if vform == 0:
            ax.plot(step_vr[0], step_vr[1], color=color, label='Respon Step')
        elif vform == 1:
            ax.plot(impl_vr[0], impl_vr[1], color=color, label='Respon Impuls')
        ax.grid()
        ax.legend()
    elif vo=='v_L1':
        sys_vl = signal.TransferFunction(num2, den)
        step_vl = signal.step(sys_vl)
        impl_vl = signal.impulse(sys_vl)
        #Plot respon
        if vform == 0:
            ax.plot(step_vl[0], step_vl[1], color=color, label='Respon Step')
        elif vform == 1:
            ax.plot(impl_vl[0], impl_vl[1], color=color, label='Respon Impuls')
        ax.grid()
        ax.legend()

In [7]:
ui_el = widgets.VBox([vo_select, R1_slider, R2_slider, C1_slider, L1_slider, vi_select, color_select1])
out_el = widgets.interactive_output(plot_electric, {'vo':vo_select,'R1':R1_slider,'R2':R2_slider,'C1':C1_slider,'L1':L1_slider,'vform':vi_select,'color':color_select1})
int_el = widgets.HBox([ui_el, out_el])

In [8]:
display(int_el)

### Analisis###

<h4>a. Respon Step </h4>

Dari hasil simulasi, didapatkan pengaruh perubahan-perubahan nilai parameter pada *output* sistem setelah diberikan *input* berupa sinyal *step*, di antaranya:
1. Kenaikan nilai $R_1$ akan menurunkan *steady-state gain* ($K$) sistem. Ini terlihat dari turunnya nilai *output* $v_{R2}$ pada keadaan *steady-state* dan turunnya nilai *maximum overshoot* ($M_p$) pada *output* $v_{L1}$. Perubahan nilai $R_1$ juga berbanding terbalik dengan perubahan koefisien redaman $\xi$, terlihat dari semakin jelas terlihatnya osilasi seiring dengan kenaikan nilai $R_1$. Perubahan nilai $R_1$ juga sebanding dengan perubahan nilai *settling time* ($t_s$). Ini terlihat dengan bertambahnya waktu sistem untuk mencapai nilai dalam rentang 2-5% dari nilai keadaan *steady-state*. 
2. Kenaikan nilai $R_2$ akan meningkatkan *steady-state gain* ($K$) sistem dengan *output* $v_{R2}$ tetapi menurunkan *steady-state gain* ($K$) *output* $v_{L1}$. Selain itu, dapat terlihat juga bahwa perubahan nilai $R_2$ berbanding terbalik dengan nilai *settling time* ($t_s$); Saat nilai $R_2$ naik, sistem mencapai kondisi *steady-state* dalam waktu yang lebih singkat. Kenaikan nilai $R_2$ juga menyebabkan penurunan nilai *maximum overshoot* ($M_p$).
3. Perubahan nilai $C_1$ sebanding dengan perubahan nilai *settling time*, seperti yang dapat terlihat dengan bertambahnya waktu yang diperlukan sistem untuk mendekati keadaan *steady-state* seiring dengan kenaikan nilai $C_1$. Selain itu nilai $C_1$ juga berbanding terbalik dengan nilai *maximum overshoot*, ini dapat dilihat dari turunnya nilai *maximum overshoot* ketika nilai $C_1$ dinaikan. Pada saat nilai $C_1$, naik, juga terlihat kenaikan nilai *delay time* ($t_d$), *rise time* ($t_r$), dan *peak time* ($t_p$).
4. Kenaikan nilai $L_1$ mengakibatkan berkurangnya nilai frekuensi osilasi, serta meningkatkan *settling time* sistem. Perubahan nilai $L_1$ ini juga sebanding dengan *steady-state gain* sistem untuk *output* $v_{L1}$.

<h4>b. Respon Impuls </h4>

Dari hasil simulasi, didapatkan pengaruh perubahan-perubahan nilai parameter pada *output* sistem setelah diberikan *input* berupa sinyal *impulse*, di antaranya:
1. Perubahan nilai $R_1$ berbanding terbalik dengan nilai *peak response*. Kenaikan nilai $R_1$ juga menaikkan nilai *settling time* ($t_s$).
2. Kenaikan nilai $R_2$ memengaruhi nilai *peak response* $v_{R2}$, tetapi tidak berpengaruh pada *peak response* $v_{L1}$. Naiknya nilai $R_2$ juga menurunkan nilai *settling time* ($t_s$), yang terlihat dari semakin cepatnya sistem mencapai kondisi *steady-state*.
3. Kenaikan nilai $C_1$ menyebabkan turunnya nilai *peak response*. Kenaikan nilai $C_1$ juga menyebabkan kenaikan nilai *settling time* ($t_s$), yang dapat dilihat dengan bertambahnya waktu yang diperlukan sistem untuk mendekati keadaan *steady-state*.
4. Kenaikan nilai $L_1$ menyebabkan turunnya nilai *peak response*. Kenaikan nilai $L_1$ juga menurunkan nilai *settling time* ($t_s$), yang dapat dilihat dari bertambahnya waktu yang diperlukan sistem untuk mendekati keadaan *steady-state*.

## 2. Pemodelan Sistem Elektromekanik
### DC Brushed Motor Torsi Besar dengan Motor Driver

Sistem yang akan dimodelkan berupa motor driver high current BTS7960 seperti gambar pertama, dihubungkan dengan motor torsi besar dengan brush pada gambar kedua.
<div>
<img src="./1.jpg" style="width:20%" align="middle">
</div>
<div>
<img src="./2.jpg" style="width:20%" align="middle">
</div>
<p style="text-align:center"><b>Sumber gambar: KRTMI URO ITB</b></p>

### Deskripsi Sistem

1. Input <br>
Sistem ini memiliki input sinyal $V_{in}$, yang merupakan fungsi waktu $V_{in}(t)$. Tegangan $v_i(t)$ ini dapat berbentuk fungsi step, impuls, atau pulse width modulation dengan duty cycle tertentu (luaran mikrokontroller umum). <br>
2. Output <br>
Sistem ini memiliki *output* posisi sudut $\theta$, kecepatan sudut motor $\omega$, percepatan sudut motor $\alpha$, dan torsi $T$. Output ditentukan sesuai kebutuhan untuk manuver robot. Terlihat variable output bergantung pada $\theta$, $\frac {d\theta}{dt}$, dan $\frac{d^2\theta}{dt}$, sehingga dicari beberapa persamaan diferensial sesuai tiap output.
3. Parameter <br>
Sistem memiliki parameter $J,K_f,K_a,L,R,K_{emf},K_{md}$ yang diturunkan dari karakteristik subsistem mekanik dan elektrik sebagai berikut.

#### Subsistem Motor Driver
Pertama ditinjau struktur sistem dari motor driver. Motor driver yang digunakan adalah tipe MOSFET BTS7960 sehingga memiliki karakteristik dinamik yang meningkat hampir dengan instant. MOSFET dirangkai sedemikian rupa sehingga dapat digunakan untuk kontrol maju/mundur motor. Diasumsikan rise-time MOSFET cukup cepat relatif terhadap sinyal dan motor driver cukup linear, maka motor driver dapat dimodelkan sebagai sistem orde 0 dengan gain sebesar $ K_{md} $. 
<img src="./4.png" style="width:30%" align="middle">
<p style="text-align:center"><b>Sumber gambar: Datasheet BTS7960</b></p>
<img src="./5.png" style="width:30%" align="middle">
<p style="text-align:center"><b>Model Orde 0 Motor Driver</b></p>
Maka persamaan dinamik output terhadap input dalam motor driver adalah <br>
$ V_m=K_{md}V_{in} $<br>
Sama seperti hubungan input output pada karakteristik statik.

#### Subsistem Motor
Lalu ditinjau struktur sistem dari motor torsi besar dengan inertia beban yang tidak dapat diabaikan.
<img src="./3.png" style="width:30%" align="middle">
<p style="text-align:center"><b>Sumber gambar: https://www.researchgate.net/figure/The-structure-of-a-DC-motor_fig2_260272509</b></p>
<br>
Maka dapat diturunkan persamaan diferensial untuk sistem mekanik.
<br>
<img src="./6.png" style="width:30%">
<img src="./7.png" style="width:30%">
<p style="text-align:center"><b>Sumber gambar: Chapman - Electric Machinery Fundamentals 4th Edition</b></p>
$$ 
T=K_a i_a 
$$ 
dengan $T$ adalah torsi dan $K_a$ adalah konstanta proporsionalitas torsi (hasil perkalian K dengan flux) untuk arus armature $i_a$.
$$
V_{emf}=K_{emf} \omega
$$
dengan $V_{emf}$ adalah tegangan penyebab electromotive force dan $K_{emf}$ konstanta proporsionalitas tegangan emf (hasil perkalian K dengan flux pada kondisi ideal tanpa voltage drop) untuk kecepatan putar sudut dari motor.
<br>
Namun, akibat terbentuknya torsi adalah berputarnya beban dengan kecepatan sudut sebesar $\omega$ dan percepatan sudut sebesar $\alpha$. Faktor proporsionalitas terhadap percepatan sudut adalah $J$ (Inersia Putar) dan terhadap kecepatan sudut sebesar $ K_f $ (Konstanta Redam Putar) Sehingga dapat diturunkan persamaan diferensial sebagai berikut (Persamaan 1):
<br>
$$ 
J\alpha + K_f\omega = T 
$$
$$
J\frac {d^2\theta}{dt} + K_f\frac {d\theta}{dt} = K_a i_a 
$$
$$ 
J\frac {d\omega}{dt} + K_f \omega = K_a i_a 
$$
Kemudian diturunkan persamaan diferensial untuk sistem elektrik yang terdapat pada motor sehingga $i_a$ dapat disubstitusi dengan input $V_{in}$ (Persamaan 2):
$$ 
L \frac{d{i_a}}{dt} + R i_a + K_{emf} \omega = V_m 
$$
$$
V_m = K_{md} V_{in}
$$
$$ 
L \frac{d{i_a}}{dt} + R i_a + K_{emf} \omega = K_{md} V_{in} 
$$

### Pemodelan dengan Fungsi Transfer
Dengan persamaan subsistem tersebut, dapat dilakukan pemodelan fungsi transfer sistem dengan transformasi ke domain laplace (s). Dilakukan penyelesaian menggunakan fungsi transfer dalam domain laplace, pertama dilakukan transfer ke domain laplace dengan asumsi
<br>
$ i_a (0) = 0 $
<br>
$ \frac {di_a}{dt} = 0 $
<br>
$ \theta (0) = 0 $
<br>
$ \omega (0) = 0 $
<br>
$ \alpha (0) = 0 $
<br>
Tidak diasumsikan terdapat voltage drop karena telah di akumulasi di $K_{emf}$, namun diasumsikan voltage drop berbanding lurus terhadap $\omega$.
<br>
Persamaan 1 menjadi:
$$
J s \omega + K_f \omega = K_a i_a
$$
Persamaan 2 menjadi:
$$
L s i_a + R i_a + K_{emf} \omega = K_{md} V_{in}
$$
$$
i_a=\frac {K_{md} V_{in}-K_{emf} \omega}{L s + R}
$$
Sehingga terbentuk fungsi transfer sistem keseluruhan dalam $\omega$ adalah:
$$
J s \omega + K_f \omega = \frac {K_a(K_{md} V_{in} - K_{emf} \omega)}{L s + R}
$$
Fungsi transfer untuk $\omega$ adalah:
$$
\omega = \frac {K_a(K_{md} V_{in}-K_{emf} \omega)}{(L s + R)(J s + K_f)}
$$
$$
\omega = \frac {K_a K_{md} V_{in}}{(L s + R)(J s + K_f)(1  + \frac {K_a K_{emf}}{(L s + R)(J s + K_f)})}
$$
$$
\frac {\omega (s)}{V_{in}(s)} = \frac {K_a K_{md}}{(L s + R)(J s + K_f)+ K_a K_{emf}}
$$
Dapat diturunkan fungsi transfer untuk theta dengan mengubah variable pada persamaan 1:
$$
J s^2 \theta + K_f s \theta = K_a i_a
$$
Persamaan 2:
$$
L s i_a + R i_a + K_{emf} s \theta = K_{md} V_{in}
$$
$$
i_a=\frac {K_{md} V_{in}-K_{emf} s \theta}{L s + R}
$$
Sehingga terbentuk fungsi transfer sistem keseluruhan dalam $\theta$ adalah:
$$
J s^2 \theta + K_f s \theta = \frac {K_a(K_{md} V_{in}-K_{emf} s \theta)}{L s + R}
$$
Fungsi transfer untuk $\theta$ adalah:
$$
\theta = \frac {K_a(K_{md} V_{in}-K_{emf} s \theta)}{(L s + R)(J s^2 + K_f s )}
$$
$$
\theta + \frac {K_a K_{emf} s \theta}{(L s + R)(J s^2 + K_f s )}= \frac {K_a K_{md} V_{in}}{(L s + R)(J s^2 + K_f s )}
$$
$$
\theta= \frac {K_a K_{md} V_{in}}{(L s + R)(J s^2 + K_f s )(1 + \frac {K_a K_{emf} s}{(L s + R)(J s^2 + K_f s )})}
$$
$$
\frac {\theta (s)}{V_{in}(s)}= \frac {K_a K_{md}}{(L s + R)(J s^2 + K_f s )+ K_a K_{emf} s}
$$
Terlihat bahwa fungsi transfer untuk $\omega$ dan $\theta$ hanya berbeda sebesar $ \frac {1}{s} $ sesuai dengan hubungan
$$
\omega = s \theta
$$
Sehingga fungsi transfer untuk $\alpha$ akan memenuhi
$$
\alpha = s\omega = s^2 \theta
$$
Sehingga fungsi transfer untuk $\alpha$ adalah:
$$
\frac {\alpha (s)}{V_{in}(s)} = \frac {K_a K_{md} s}{(L s + R)(J s + K_f)+ K_a K_{emf}}
$$

### Output
Dari fungsi transfer, diformulasikan persamaan output posisi sudut $\theta$, kecepatan sudut motor $\omega$, percepatan sudut $\alpha$, dan torsi $T$ dalam fungsi waktu (t).
$$
\theta (t) = \mathscr {L^{-1}} \{\frac {K_a K_{md} V_{in}(s)}{(L s + R)(J s^2 + K_f s )+ K_a K_{emf} s}\}
$$
<br>
$$
\omega (t) = \mathscr {L^{-1}} \{\frac {K_a K_{md} V_{in}(s)}{(L s + R)(J s + K_f)+ K_a K_{emf}}\}
$$
<br>
$$
\alpha (t)= \mathscr {L^{-1}} \{\frac {K_a K_{md} Vin_{in}(s) s}{(L s + R)(J s + K_f)+ K_a K_{emf}}\}
$$
<br>
$$
T = \frac {K_a(K_{md} V_{in}-K_{emf} \omega)}{L s + R} 
$$

In [9]:
# Digunakan penyelesaian numerik untuk output
import numpy as np
from scipy.integrate import odeint
import scipy.signal as sig
import matplotlib.pyplot as plt
from sympy.physics.mechanics import dynamicsymbols, SymbolicSystem
from sympy import *
import control as control

In [10]:
vin = symbols ('V_{in}') #import symbol input

In [11]:
omega, theta, alpha = dynamicsymbols('omega theta alpha') #import symbol output

In [12]:
ka,kmd,l,r,j,kf,kemf,s,t = symbols ('K_a K_{md} L R J K_f K_{emf} s t')#import symbol parameter dan s

In [13]:
thetaOverVin = (ka*kmd)/((l*s+r)*(j*s**2+kf*s)+ka*kemf*s) #persamaan fungsi transfer theta
polyThetaOverVin = thetaOverVin.as_poly() #Penyederhanaan persamaan
polyThetaOverVin

Poly((1/(J*L*s**3 + J*R*s**2 + K_a*K_{emf}*s + K_f*L*s**2 + K_f*R*s))*K_a*K_{md}, 1/(J*L*s**3 + J*R*s**2 + K_a*K_{emf}*s + K_f*L*s**2 + K_f*R*s), K_a, K_{md}, domain='ZZ')

In [14]:
omegaOverVin = (ka*kmd)/((l*s+r)*(j*s+kf)+ka*kemf) #persamaan fungsi transfer omega
polyOmegaOverVin = omegaOverVin.as_poly() #Penyederhanaan persamaan
polyOmegaOverVin

Poly((1/(J*L*s**2 + J*R*s + K_a*K_{emf} + K_f*L*s + K_f*R))*K_a*K_{md}, 1/(J*L*s**2 + J*R*s + K_a*K_{emf} + K_f*L*s + K_f*R), K_a, K_{md}, domain='ZZ')

In [15]:
alphaOverVin = (ka*kmd*s)/((l*s+r)*(j*s+kf)+ka*kemf)
polyAlphaOverVin = alphaOverVin.as_poly() #Penyederhanaan persamaan
polyAlphaOverVin

Poly(s*(1/(J*L*s**2 + J*R*s + K_a*K_{emf} + K_f*L*s + K_f*R))*K_a*K_{md}, s, 1/(J*L*s**2 + J*R*s + K_a*K_{emf} + K_f*L*s + K_f*R), K_a, K_{md}, domain='ZZ')

In [16]:
torqueOverVin= ka*(kmd-kemf*((ka*kmd)/((l*s+r)*(j*s+kf)+ka*kemf)))/(l*s+r) #Penyederhanaan persamaan torsi
polyTorqueOverVin = torqueOverVin.as_poly()
polyTorqueOverVin

Poly(-(1/(J*L**2*s**3 + 2*J*L*R*s**2 + J*R**2*s + K_a*K_{emf}*L*s + K_a*K_{emf}*R + K_f*L**2*s**2 + 2*K_f*L*R*s + K_f*R**2))*K_a**2*K_{emf}*K_{md} + (1/(L*s + R))*K_a*K_{md}, 1/(J*L**2*s**3 + 2*J*L*R*s**2 + J*R**2*s + K_a*K_{emf}*L*s + K_a*K_{emf}*R + K_f*L**2*s**2 + 2*K_f*L*R*s + K_f*R**2), 1/(L*s + R), K_a, K_{emf}, K_{md}, domain='ZZ')

In [17]:
def plot_elektromekanik(Ka,Kmd,L,R,J,Kf,Kemf,VinType,tMax,dutyCycle,grid):
    # Parameter diberi value dan model system dibentuk dalam transfer function yang dapat diolah python
    Ka = Ka
    Kmd = Kmd
    L = L
    R = R
    J = J
    Kf = Kf
    Kemf = Kemf
    # Pembuatan model transfer function
    tf = control.tf
    tf_Theta_Vin = tf([Ka*Kmd],[J*L,(J*R+Kf*L),(Ka*Kemf+Kf*R),0])
    tf_Omega_Vin = tf([Ka*Kmd],[J*L,(J*R+Kf*L),(Ka*Kemf+Kf*R)])
    tf_Alpha_Vin = tf([Ka*Kmd,0],[J*L,(J*R+Kf*L),(Ka*Kemf+Kf*R)])
    tf_Torque_Vin = tf([Ka*Kmd],[L,R]) - tf([Kmd*Kemf*Ka**2],[J*L**2,(2*J*L*R+Kf*L**2),(J*R**2+Ka*Kemf*L+2*Kf*L*R),(Ka*Kemf*R+Kf*R**2)])
    f, axs = plt.subplots(4, sharex=True, figsize=(10, 10))
    # Fungsi mengatur rentang waktu analisis (harus memiliki kelipatan 1 ms)
    def analysisTime(maxTime):
        ts=np.linspace(0, maxTime, maxTime*100)
        return ts
    t=analysisTime(tMax)
    if VinType== 2:
        # Input pwm dalam 1 millisecond
        def Pwm(dutyCycle,totalTime):
            trepeat=np.linspace(0, 1, 100)
            squareWave=(5*sig.square(2 * np.pi * trepeat, duty=dutyCycle))
            finalInput=np.zeros(len(totalTime))
            for i in range(len(squareWave)):
                if squareWave[i]<0:
                    squareWave[i]=0
            for i in range(len(totalTime)):
                finalInput[i]=squareWave[i%100]
            return finalInput
        pwm=Pwm(dutyCycle,t)
        tPwmTheta, yPwmTheta, xPwmTheta = control.forced_response(tf_Theta_Vin, T=t, U=pwm, X0=0)
        tPwmOmega, yPwmOmega, xPwmOmega = control.forced_response(tf_Omega_Vin, t, pwm, X0=0)
        tPwmAlpha, yPwmAlpha, xPwmAlpha = control.forced_response(tf_Alpha_Vin, t, pwm, X0=0)
        tPwmTorque, yPwmTorque, xPwmTorque = control.forced_response(tf_Torque_Vin, t, pwm, X0=0)
        axs[0].plot(tPwmTheta, yPwmTheta, color = 'blue', label ='Theta')
        axs[1].plot(tPwmOmega, yPwmOmega, color = 'red', label ='Omega')
        axs[2].plot(tPwmAlpha, yPwmAlpha, color = 'black', label ='Alpha')
        axs[3].plot(tPwmTorque, yPwmTorque, color = 'green', label ='Torque')
        axs[0].title.set_text('Theta $(rad)$ (Input PWM)')
        axs[1].title.set_text('Omega $(\\frac {rad}{ms})$ (Input PWM)')
        axs[2].title.set_text('Alpha $(\\frac {rad}{ms^2})$ (Input PWM)')
        axs[3].title.set_text('Torque $(Nm)$ (Input PWM)')
    elif VinType== 0:
        tStepTheta, yStepTheta = control.step_response(tf_Theta_Vin,T=t, X0=0)
        tStepOmega, yStepOmega = control.step_response(tf_Omega_Vin,T=t, X0=0)
        tStepAlpha, yStepAlpha = control.step_response(tf_Alpha_Vin,T=t, X0=0)
        tStepTorque, yStepTorque = control.step_response(tf_Torque_Vin, T=t, X0=0)
        axs[0].plot(tStepTheta, yStepTheta, color = 'blue', label ='Theta')
        axs[1].plot(tStepOmega, yStepOmega, color = 'red', label ='Omega')
        axs[2].plot(tStepAlpha, yStepAlpha, color = 'black', label ='Alpha')
        axs[3].plot(tStepTorque, yStepTorque, color = 'green', label ='Torque')
        axs[0].title.set_text('Theta $(rad)$ (Input Step)')
        axs[1].title.set_text('Omega $(\\frac {rad}{ms})$ (Input Step)')
        axs[2].title.set_text('Alpha $(\\frac {rad}{ms^2})$(Input Step)')
        axs[3].title.set_text('Torque $(Nm)$ (Input Step)')
    elif VinType== 1 :
        tImpulseTheta, yImpulseTheta = control.impulse_response(tf_Theta_Vin,T=t, X0=0)
        tImpulseOmega, yImpulseOmega = control.impulse_response(tf_Omega_Vin,T=t, X0=0)
        tImpulseAlpha, yImpulseAlpha = control.impulse_response(tf_Alpha_Vin,T=t, X0=0)
        tImpulseTorque, yImpulseTorque = control.impulse_response(tf_Torque_Vin, T=t, X0=0)
        axs[0].plot(tImpulseTheta, yImpulseTheta, color = 'blue', label ='Theta')
        axs[1].plot(tImpulseOmega, yImpulseOmega, color = 'red', label ='Omega')
        axs[2].plot(tImpulseAlpha, yImpulseAlpha, color = 'black', label ='Alpha')
        axs[3].plot(tImpulseTorque, yImpulseTorque, color = 'green', label ='Torque')
        axs[0].title.set_text('Theta $(rad)$ (Input Impulse)')
        axs[1].title.set_text('Omega $(\\frac {rad}{ms})$ (Input Impulse)')
        axs[2].title.set_text('Alpha $(\\frac {rad}{ms^2})$ (Input Impulse)')
        axs[3].title.set_text('Torque $(Nm)$ (Input Impulse)')
    axs[0].legend()
    axs[1].legend()
    axs[2].legend()
    axs[3].legend()
    axs[0].grid(grid)
    axs[1].grid(grid)
    axs[2].grid(grid)
    axs[3].grid(grid)

In [18]:
#DEFINISI WIDGETS PARAMETER
Ka_slider = widgets.FloatSlider(
    value=0.1,
    min=0.1,
    max=30.0,
    step=0.1,
    description='$K_a (\\frac {Nm}{A})$',
    layout=Layout(width='80%', height='50px'),
    style={'description_width': '200px'},
)
Kmd_slider = widgets.FloatSlider(
    value=0.1,
    min=2,
    max=20.0,
    step=0.1,
    description='$K_{md} (\\frac {V}{V})$',
    layout=Layout(width='80%', height='50px'),
    style={'description_width': '200px'},
)
L_slider = widgets.FloatSlider(
    value=0.1,
    min=0.1,
    max=100.0,
    step=0.1,
    description='$L (mH)$',
    layout=Layout(width='80%', height='50px'),
    style={'description_width': '200px'},
)
R_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=1000,
    step=1,
    description='$R (\Omega)$',
    layout=Layout(width='80%', height='50px'),
    style={'description_width': '200px'},
)
J_slider = widgets.FloatSlider(
    value=0.1,
    min=0.1,
    max=100.0,
    step=0.1,
    description='$J (\\frac {Nm(ms)^2}{rad})$',
    layout=Layout(width='80%', height='50px'),
    style={'description_width': '200px'},
)
Kf_slider = widgets.FloatSlider(
    value=0.1,
    min=0.1,
    max=100.0,
    step=0.1,
    description='$K_{f} (\\frac {Nm(ms)}{rad})$',
    layout=Layout(width='80%', height='50px'),
    style={'description_width': '200px'},
)
Kemf_slider = widgets.FloatSlider(
    value=0.1,
    min=0.1,
    max=30,
    step=0.1,
    description='$K_{emf} (\\frac {V(ms)}{rad})$',
    layout=Layout(width='80%', height='50px'),
    style={'description_width': '200px'},
)
VinType_select = widgets.Dropdown(
    options=[('Step', 0), ('Impulse', 1),('PWM',2)],
    description='Tipe Sinyal Input:',
    layout=Layout(width='80%', height='50px'),
    style={'description_width': '200px'},
)
tMax_slider = widgets.IntSlider(
    value=10,
    min=1,
    max=500,
    step=1,
    description='$t_{max} (ms)$',
    layout=Layout(width='80%', height='50px'),
    style={'description_width': '200px'},
)
dutyCycle_slider = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1.0,
    step=0.05,
    description='$Duty Cycle (\%)$',
    layout=Layout(width='80%', height='50px'),
    style={'description_width': '200px'},
)
grid_button = widgets.ToggleButton(
    value=True,
    description='Grid',
    icon='check',
    layout=Layout(width='20%', height='50px',margin='10px 10px 10px 350px'),
    style={'description_width': '200px'},
)
ui_em = widgets.VBox([Ka_slider,Kmd_slider,L_slider,R_slider,J_slider,Kf_slider,Kemf_slider,VinType_select,tMax_slider,dutyCycle_slider,grid_button])
out_em = widgets.interactive_output(plot_elektromekanik, {'Ka':Ka_slider,'Kmd':Kmd_slider,'L':L_slider,'R':R_slider,'J':J_slider,'Kf':Kf_slider,'Kemf':Kemf_slider,'VinType':VinType_select,'tMax':tMax_slider,'dutyCycle':dutyCycle_slider, 'grid':grid_button})

In [19]:
display(ui_em,out_em)

Output()

### Analisis

Karena model memiliki persamaan yang cukup kompleks sehingga tidak dapat diambil secara intuitive kesimpulan parameter terhadap output sistem, akan dilakukan percobaan menggunakan slider untuk mengubah parameter dan mengamati interaksi perubahan antara parameter. Akan dilakukan juga perubahan bentuk input dan analisa efek penggunaan PWM sebagai modulasi sinyal step dengan besar maksimum 5V terhadap output.
#### 1. Peningkatan $K_a$
Peningkatan $K_a$ menyebabkan peningkatan osilasi ($\omega_d$) dan meningkatkan gain pada output $\omega$ dan $\alpha$ serta meningkatkan gradien dari output $\theta$. Namun, gain Torque tidak terpengaruh.
#### 2. Peningkatan $K_{md}$
Peningkatan $K_{md}$ membuat amplitudo $V_{in}$ meningkat sehingga amplitudo output bertambah.
#### 3. Peningkatan $L$
Peningkatan $L$ menyebabkan peningkatan kecepatan sudut $\omega$ dan $T$ menjadi lebih lambat serta penurunan $\alpha$ yang semakin lambat sehingga menyebabkan peningkatan $\theta$ semakin lambat (peningkatan rise time).
#### 4. Peningkatan $R$
Peningkatan $R$ menyebabkan osilasi output ($\omega_d$) $\omega$, $\alpha$, dan Torque semakin kecil dan gain yang semakin kecil sehingga mengurangi gradien dari output $\theta$.
#### 5. Peningkatan $J$
Peningkatan $J$ meningkatkan gain Torque dan menurunkan gain $\theta$, $\omega$, dan $\alpha$. 
#### 6. Peningkatan $K_f$
Peningkatan $K_f$ meningkatkan gain Torque dan menurunkan gain $\theta$, $\omega$, dan $\alpha$.
#### 7. Peningkatan $K_{emf}$
Peningkatan $K_{emf}$ menurunkan gain Torque, $\theta$, $\omega$, dan $\alpha$.
#### 8. Interaksi antar parameter
Perbandingan pengurangan $R$ dibanding peningkatan $K_a$ kira kira 3 kali lipat. Peningkatan pada $J$ dan $K_f$ terbatas pada peningkatan $K_a$. Secara fisis, peningkatan $K_a$ dan $K_{emf}$ terjadi secara bersamaan dan hampir sebanding (hanya dibedakan pada voltage drop pada berbagai komponen), diikuti oleh $L$ sehingga untuk $K_a$ dan $K_{emf}$ besar, waktu mencapai steady state juga semakin lama. Hal yang menarik adalah $K_a$ dan $K_{emf}$ membuat sistem memiliki gain (transfer energi) yang kecil jika hanya ditinjau dari peningkatan nilai $K_a$ dan $K_{emf}$, namun ketika diikuti peningkatan $V_{in}$ sistem memiliki transfer energi yang lebih besar daripada sebelumnya pada keadaan steady state. Jadi dapat disimpulkan bahwa $K_a$ dan $K_{emf}$ harus memiliki nilai yang cukup besar agar konfigurasi sesuai dengan input $V_{in}$ dan menghasilkan transfer energi yang efisien. Input $V_{in}$ juga harus sesuai dengan sistem $K_a$ dan $K_{emf}$ yang ada sehingga dapat memutar motor (ini mengapa terdapat voltage minimum dan voltage yang disarankan untuk menjalankan sebuah motor).
#### 9. Pengaruh Input Step
Penggunaan input step memiliki osilasi ($\omega_d$) semakin sedikit.
#### 10. Pengaruh Input Impuls
Penggunaan input impulse membuat $\theta$ mencapai steady state karena motor berhenti berputar sehingga $\omega$,$\alpha$, dan Torque memiliki nilai steady state 0.
#### 11. Pengaruh Input PWM
Penggunaan input PWM dengan duty cycle tertentu membuat osilasi yang semakin banyak, namun dengan peningkatan duty cycle, osilasi semakin sedikit (semakin mendekati sinyal step). Hal yang menarik disini adalah sinyal PWM dapat digunakan untuk mengontrol, tetapi ketika tidak digunakan pengontrol, sinyal PWM malah memberikan osilasi pada sistem.

## 3. Pemodelan Sistem Mekanik
Dimodelkan sistem mekanik sebagai berikut
<img src="./10.png" style="width:20%">
<p style="text-align: center"><b>Sistem Mekanik Sederhana dengan Bond Graph</b></p>

### Deskripsi Sistem
1. Input
$F$ sebagai gaya yang dikerjakan pada massa
2. Output
$x$ sebagai perpindahan, $v$ sebagai kecepatan, dan $a$ sebagai percepatan pada massa
3. Parameter
Dari penurunan bond graph, didapatkan parameter $k$, $b$, dan $m$

### Pemodean Transfer Function
Fungsi transfer dapat dengan mudah di turunkan dari hubungan bond graph
$$
m \frac {d^2 x}{dt^2} = F-kx-b\frac{dx}{dt}
$$
<br>
Transformasi laplace menghasilkan
<br>
$$
s^2 x = \frac {F}{m}-x\frac {k}{m}-sx\frac{b}{m}
$$
$$
(s^2+s\frac{b}{m}+\frac {k}{m})x=\frac {F}{m}
$$
<br>
Untuk x:
<br>
$$
\frac {x}{F}=\frac {1}{(ms^2+bs+k)}
$$
<br>
Untuk v:
<br>
$$
\frac {v}{F}=\frac {s}{(ms^2+bs+k)}
$$
<br>
Untuk a:
<br>
$$
\frac {a}{F}=\frac {s^2}{(ms^2+bs+k)}
$$

In [20]:
# Digunakan penyelesaian numerik untuk output
import numpy as np
from scipy.integrate import odeint
import scipy.signal as sig
import matplotlib.pyplot as plt
from sympy.physics.mechanics import dynamicsymbols, SymbolicSystem
from sympy import *
import control as control

In [50]:
def plot_mekanik(M,B,K,VinType,grid):
    # Parameter diberi value dan model system dibentuk dalam transfer function yang dapat diolah python
    m=M
    b=B
    k=K
    tf = sig.TransferFunction
    tf_X_F=tf([1],[m,b,k])
    tf_V_F=tf([1,0],[m,b,k])
    tf_A_F=tf([1,0,0],[m,b,k])
    f, axs = plt.subplots(3, sharex=True, figsize=(10, 10))
    if VinType==0:
        tImpX,xOutImp=sig.impulse(tf_X_F)
        tImpV,vOutImp=sig.impulse(tf_V_F)
        tImpA,aOutImp=sig.impulse(tf_A_F)
        axs[0].plot(tImpX,xOutImp, color = 'blue', label ='x')
        axs[1].plot(tImpV,vOutImp, color = 'red', label ='v')
        axs[2].plot(tImpA,aOutImp, color = 'green', label ='a')
        axs[0].title.set_text('Perpindahan Linear $(m)$ (Input Impuls)')
        axs[1].title.set_text('Kecepatan Linear $(\\frac {m}{s})$ (Input Impuls)')
        axs[2].title.set_text('Percepatan Linear $(\\frac {m}{s^2})$ (Input Impuls)')
    elif VinType==1:
        tStepX,xOutStep=sig.step(tf_X_F)
        tStepV,vOutStep=sig.step(tf_V_F)
        tStepA,aOutStep=sig.step(tf_A_F)
        axs[0].plot(tStepX,xOutStep, color = 'blue', label ='x')
        axs[1].plot(tStepV,vOutStep, color = 'red', label ='v')
        axs[2].plot(tStepA,aOutStep, color = 'green', label ='a')
        axs[0].title.set_text('Perpindahan Linear $(m)$ (Input Step)')
        axs[1].title.set_text('Kecepatan Linear $(\\frac {m}{s})$ (Input Step)')
        axs[2].title.set_text('Percepatan Linear $(\\frac {m}{s^2})$ (Input Step)')
    axs[0].legend()
    axs[1].legend()
    axs[2].legend()
    axs[0].grid(grid)
    axs[1].grid(grid)
    axs[2].grid(grid)

In [51]:
M_slider = widgets.FloatSlider(
    value=0.1,
    min=0.1,
    max=30.0,
    step=0.1,
    description='Massa $(kg)$',
    layout=Layout(width='80%', height='50px'),
    style={'description_width': '200px'},
)
B_slider = widgets.FloatSlider(
    value=0.1,
    min=2,
    max=20.0,
    step=0.1,
    description='Konstanta Redaman $(\\frac {Ns}{m})$',
    layout=Layout(width='80%', height='50px'),
    style={'description_width': '200px'},
)
K_slider = widgets.FloatSlider(
    value=0.1,
    min=0.1,
    max=100.0,
    step=0.1,
    description='Konstanta pegas $(\\frac {N}{m})$',
    layout=Layout(width='80%', height='50px'),
    style={'description_width': '200px'},
)
VinType_select = widgets.Dropdown(
    options=[('Impulse', 0), ('Step', 1)],
    description='Tipe Sinyal Input:',
    layout=Layout(width='80%', height='50px'),
    style={'description_width': '200px'},
)
grid_button = widgets.ToggleButton(
    value=True,
    description='Grid',
    icon='check',
    layout=Layout(width='20%', height='50px',margin='10px 10px 10px 350px'),
    style={'description_width': '200px'},
)
ui_mk = widgets.VBox([M_slider,B_slider,K_slider,VinType_select,grid_button])
out_mk = widgets.interactive_output(plot_mekanik, {'M':M_slider,'B':B_slider,'K':K_slider,'VinType':VinType_select,'grid':grid_button})

In [52]:
display(ui_mk,out_mk)

Output()

### Analisis
Berdasarkan persamaan yang cukup sederhana, sistem mekanik orde dua memiliki karakteristik berikut:
1. Pengaruh peningkatan massa
Massa pada sistem berperilaku seperti komponen inersial yang meningkatkan rise time dan settling time ketika diperbesar.
2. Pengaruh peningkatan konstanta redaman
Konstanta redaman berperilaku seperti komponen hambatan yang meredam sistem sehingga maximum overshoot menjadi kecil (akibat peningkatan damping ratio) ketika peningkatan konstanta redaman terjadi. Konstanta redaman juga berpengaruh pada settling time, dimana peningkatan konstanta redaman meningkatkan settling time.
3. Pengaruh peningkatan konstanta pegas
Konstanta pegas berperilaku seperti komponen kapasitansi yang mengurangi besar gain dari perpindahan, mengurangi damping ratio, meningkatkan frekuensi osilasi sistem, mengurangi amplitudo kecepatan sistem, mempercepat settling time, dan mempercepat peak time, meningkatkan maximum overshoot.

## 3. Pemodelan Sistem Termofluidik
Akan dimodelkan sistem termofluidik sederhana berupa tangki dengan air mengalir dengan asumsi peninjauan konduksi tanpa memperhitungkan konveksi, massa jenis $\rho$ konstan, kalor jenis $C$ konstan, koefisien transfer kalor $U$ konstan, dan luas alas tangki $A$ konstan. Distribusi panas diabaikan dan dianggap heat flux tersebar secara merata dalam sistem, serta tangki dianggap isoterm. Diasumsikan juga suhu lingkungan konstan sebesar $T_{env}$
<img src="./8.jpg" style="width:20%">
<p style="text-align: center"><b>Water Heater</b></p>
<p style="text-align: center"><b>Sumber: https://en.wikipedia.org/wiki/Water_heating#/media/File:Water_Heater_White.jpg</b></p>
<img src="./9.jpg" style="width:50%">
<p style="text-align: center"><b>Diagram Sederhana Sistem</b></p>

### Deskripsi Sistem
1. Input
$q_i$ sebagai kalor oleh pemanas elektrik, $f_i$ sebagai flow masuk
2. Output
$f_o$ sebagai flow keluar, $T_{tank}$ sebagai suhu tangki
3. Parameter
didapatkan parameter 

#### Konservasi Massa
$$
\frac {dM}{dt}=f_i-f_o
$$
$$
\frac {d(\rho A h)}{dt}=f_i-f_o
$$
#### Konservasi Energi
$$
\frac {dE}{dt}=q_i - q_o
$$
#### Konservasi Massa dan Energi
$$
C \frac {\rho A  d (h T_{tank})}{dt}=q_i-UA(T_{tank}-T_{env})
$$
$$
C \rho A (T_{tank} \frac {dh}{dt} + h \frac {dT_{tank}}{dt})=q_i-UA(T_{tank}-T_{env})
$$

### Pemodelan dengan Bond Graph

### Pemodean Transfer Function